In [ ]:
import pretty_midi
import numpy as np
import muspy
import os
import csv
%run parse.ipynb
# %run mlp_model.ipynb

In [ ]:
# Install soundfont for muspy
muspy.download_musescore_soundfont()

In [ ]:
# Note:
# To synthesize, after generate the start and end timings, call storeCSV() then call the synthesizeOutput()
# ONLY synthesize one at a time

In [ ]:
# @param:
    # startTimings: an array of expressive start times
    # endTimings: an array of expressive end times
    # fileName: name of CSV file
# @return: output a CSV file containing the given expressive start and end timings
def storeCSV(startTimings, endTimings, fileName):
    if len(startTimings) != len(endTimings):
        raise ValueError("Arrays must have the same length")
    else:
        rows = zip(startTimings, endTimings)
        outputDirectory = './csv-output/'
        with open(outputDirectory + fileName + '.csv', mode='w', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(['start', 'end'])  # Write header
            writer.writerows(rows)

In [ ]:
# @param:
    # mxlPath: path to the mxl file
    # outputPath: output the .wav file to this path   
    # name: name of the .wav file
# @return: wav file in src folder that converts the original 
def synthesizeInput(mxlPath, outputPath, name):
    m = muspy.read(mxlPath)
    m.write_audio(outputPath + name + ".wav")

In [ ]:
# @param:
    # tempoPath: path to the tempo file
    # inputPath: path to the original input file -> to get pitch and velocity
    # outputPath: path to the generative output file containing the expressive start and end timings
    # name: name of the .wav file
# @return: wav file in src folder that converts the expressive timings
def synthesizeOutput(tempoPath, inputPath, outputPath, name):
    tempo = parseTempoFile(tempoPath)
    MIDI = pretty_midi.PrettyMIDI(initial_tempo = tempo)

    # Create an Instrument instance for a violin instrument
    violinProgram = pretty_midi.instrument_name_to_program('Violin')
    violin = pretty_midi.Instrument(program=violinProgram)

    # Create notes
    inputInfo = parseInputFile(inputPath)
    outputInfo = parseOutputFile(outputPath)

    for notes, times in zip(inputInfo, outputInfo):
        note = pretty_midi.Note(
            velocity=64, pitch=int(notes[2]), start=times[0], end=times[1])
        violin.notes.append(note)

    # Add the instrument to the PrettyMIDI object
    MIDI.instruments.append(violin)

    # Write to a MIDI file
    MIDI.write('violin.mid')

    # Convert to WAV using fluidsynth
    soundfontPath = '../../.muspy/musescore-general/MuseScore_General.sf3'
    outputDirectory = './audio-output/'
    os.system(f'fluidsynth -ni {soundfontPath} {"violin.mid"} -F {outputDirectory + name + ".wav"} -r 44100')

    # Remove the MIDI file
    os.remove('violin.mid')

In [ ]:
# Test case for synthesizing input
mxlPath = '../bach-violin-dataset/dataset/scores/bwv1001/bwv1001_mov1.mxl'
name = 'bwv1001_source'
synthesizeInput(mxlPath, name)

In [ ]:
# Test case for synthesizing output
tempoPath = '../bach-violin-dataset/dataset/tempos/emil-telmanyi/bwv1001/emil-telmanyi_bwv1001_mov1.txt'
inputPath = '../bach-violin-dataset/dataset/source-input/emil-telmanyi/bwv1001/emil-telmanyi_bwv1001_mov1.csv'
generatedOutputPath = '../bach-violin-dataset/dataset/ground-truth/emil-telmanyi/bwv1001/emil-telmanyi_bwv1001_mov1.csv' # using groundtruth instead of generated start, end timings
name = 'bwv1001_groundtruth'
synthesizeOutput(tempoPath, inputPath, generatedOutputPath, name)